In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Date
0,World,2399954,69188.0,164943.0,4896.0,615703.0,1619308,54215.0,308.00,21.2,0.0,0.0,2020-04-19 17:20:01.734414
1,USA,761484,22692.0,40423.0,1409.0,69956.0,651105,13556.0,2301.00,122.0,3846500.0,11621.0,2020-04-19 17:20:01.734414
2,Spain,196664,2248.0,20595.0,552.0,77357.0,98712,7371.0,4206.00,440.0,930230.0,19896.0,2020-04-19 17:20:01.734414
3,Italy,178972,3047.0,23660.0,433.0,47055.0,108257,2635.0,2960.00,391.0,1356541.0,22436.0,2020-04-19 17:20:01.734414
4,France,152894,1101.0,19718.0,395.0,36578.0,96598,5744.0,2342.00,302.0,463662.0,7103.0,2020-04-19 17:20:01.734414
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,South Sudan,4,0.0,0.0,0.0,0.0,4,0.0,0.40,0.0,0.0,0.0,2020-04-19 17:20:01.734414
209,Anguilla,3,0.0,0.0,0.0,1.0,2,0.0,200.00,0.0,0.0,0.0,2020-04-19 17:20:01.734414
210,Saint Pierre Miquelon,1,0.0,0.0,0.0,0.0,1,0.0,173.00,0.0,0.0,0.0,2020-04-19 17:20:01.734414
211,Yemen,1,0.0,0.0,0.0,0.0,1,0.0,0.03,0.0,0.0,0.0,2020-04-19 17:20:01.734414


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,...,Tests/ 1M pop,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Spain,196664,2248.0,20595.0,552.0,77357.0,98712,7371.0,4206.0,440.0,...,19896.0,2020-04-19 17:20:01.734414,-3.647550,40.244487,97952.0,0.210256,0.104722,0.125860,0.013180,Last 8 columns are added by Roaya
2,Italy,178972,3047.0,23660.0,433.0,47055.0,108257,2635.0,2960.0,391.0,...,22436.0,2020-04-19 17:20:01.734414,12.070013,42.796626,70715.0,0.334582,0.132199,0.090863,0.012012,Last 8 columns are added by Roaya
3,France,152894,1101.0,19718.0,395.0,36578.0,96598,5744.0,2342.0,302.0,...,7103.0,2020-04-19 17:20:01.734414,2.618787,47.824905,56296.0,0.350256,0.128965,0.072336,0.009329,Last 8 columns are added by Roaya
0,USA,761484,22692.0,40423.0,1409.0,69956.0,651105,13556.0,2301.0,122.0,...,11621.0,2020-04-19 17:20:01.734414,-97.922211,39.381266,110379.0,0.366220,0.053085,0.141828,0.007529,Last 8 columns are added by Roaya
177,China,82735,16.0,4632.0,0.0,77062.0,1041,85.0,57.0,3.0,...,0.0,2020-04-19 17:20:01.734414,103.819074,36.561765,81694.0,0.056699,0.055986,0.104970,0.005877,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Uganda,55,0.0,0.0,0.0,28.0,27,0.0,1.0,0.0,...,243.0,2020-04-19 17:20:01.734414,32.369080,1.274693,28.0,0.000000,0.000000,0.000036,0.000000,Last 8 columns are added by Roaya
122,Madagascar,121,1.0,0.0,0.0,39.0,82,1.0,4.0,0.0,...,85.0,2020-04-19 17:20:01.734414,46.704737,-19.371896,39.0,0.000000,0.000000,0.000050,0.000000,Last 8 columns are added by Roaya
109,Vietnam,268,0.0,0.0,0.0,203.0,65,8.0,3.0,0.0,...,2119.0,2020-04-19 17:20:01.734414,106.299147,16.646017,203.0,0.000000,0.000000,0.000261,0.000000,Last 8 columns are added by Roaya
118,Rwanda,147,3.0,0.0,0.0,76.0,71,0.0,11.0,0.0,...,537.0,2020-04-19 17:20:01.734414,29.919885,-1.990338,76.0,0.000000,0.000000,0.000098,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"USA","TotalCases":761484,"NewCases":22692.0,"TotalDeaths":40423.0,"NewDeaths":1409.0,"TotalRecovered":69956.0,"ActiveCases":651105,"CriticalCases":13556.0,"Tot\\u00a0Cases\\/1M pop":2301.0,"Deaths\\/1M pop":122.0,"TotalTests":3846500.0,"Tests\\/ 1M pop":11621.0,"Date":1587316801734,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":110379.0,"ClosedDeathRate":0.3662200237,"CurrentDeathRate":0.0530845034,"Weight":0.1418279238,"DeathRateXWeight":0.0075288649,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spain","TotalCases":196664,"NewCases":2248.0,"TotalDeaths":20595.0,"NewDeaths":552.0,"TotalRecovered":77357.0,"ActiveCases":98712,"CriticalCases":7371.0,"Tot\\u00a0Cases\\/1M pop":4206.0,"Deaths\\/1M pop":440.0,"TotalTests":930230.0,"Tests\\/ 1M pop":19896.0,"Date":1587316801734,"Longitude":-3.64755047,"Latitude":40.24448698,"ClosedCases":97952.0,"ClosedDeathRate":0.2102560438,"CurrentDeathRate":0.1047217589,"Weight":0.1258602524,"DeathRateXWeight":0.0131803